<a href="https://colab.research.google.com/github/staniya/shibaki-RF-analysis/blob/main/RF%E8%A7%A3%E6%9E%90%E3%83%87%E3%83%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
# データの図を出力するライブラリ
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

# 環境設定
import sys
sys.path.append('models')

# 実際の機械学習ライブラリ
import tensorflow as tf

In [2]:
# Google Driveからデータを引っ張ってくる
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/MyDrive/shibaki analysis/AE-FILD BAL patient_data.xlsx' patient_data.xlsx

Mounted at /content/drive


In [3]:
# openpyxlというエンジンを使用して、ExcelファイルをPandasのデータフレームとして読みこむ
!pip install openpyxl==2.6
import pandas as pd
AE_FILD_df = pd.read_excel('patient_data.xlsx', engine='openpyxl') 

     |████████████████████████████████| 172 kB 5.1 MB/s 
  Created wheel for openpyxl: filename=openpyxl-2.6.0-py2.py3-none-any.whl size=243902 sha256=e9959367097667be5a3c4a3a0a0924bcc4a798fe0676652d0ae9878ecd5add4b
  Stored in directory: /root/.cache/pip/wheels/4a/5e/2e/d6e53a8a4b88aa95509c0c94b5629a9f58bc1260d7315e6a92
Successfully built openpyxl
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 2.5.9
    Uninstalling openpyxl-2.5.9:
      Successfully uninstalled openpyxl-2.5.9


In [4]:
AE_FILD_df.head()

,No,ID,90day mortality\nsurvival:0\ndeath:1,sex\nmale:0\nfemale:1,age,smoking status\nNever:0\nEx(quit smoking over a year ago):1\nCurrent:2,Brinkman Index,emphysema\n(-):0\n(+):1,Baseline ILD\nUIP:0\nprobable UIP:1\nIndeterminate for UIP:2\nAlternative diagnosis:3,Collagen disease\n(-):0\n(+):1,IPAF\n(-):0\n(+):1,P/F ratio,LDH,CRP,KL-6,\nBAL(neutro %),\nBAL(Lymph %),\nBAL(Eos %),\nBAL(M %),\nBAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,K003,558277,0,0,78,1,1600,0,1,0,0,291.666667,345,2.35,1599,41,38,3.0,13,3.5,664.773204,13313.791597,62.176805,40.438809,9.765625,4.450383,NaN,10.963133,3.200000,10.836599,3.200000,3.200000,33.265989,3.200000,3.200000,954.746558,3.200000,34.734508,3.2,81.530691,3.2,3.2,3.2,12.667984,3.2,395.799401,3.379747,3.2,3.2,3.2,3.2,14.451335,3.20000,23.016166,3.2,313.439925,2160.940947,1009.878650,28.906316,30.940577,4.483265,3.2,3.2,4.221958
1,K022,1848827,0,1,72,0,0,0,2,0,0,264.571429,277,5.65,1620,18,21,0.0,61,3.9,561.092206,12107.253258,83.356309,64.460473,9.765625,6.932559,NaN,15.963448,3.200000,53.038298,3.200000,4.635806,75.733522,6.710735,10.849171,2849.703534,9.801179,46.135602,3.2,60.021260,3.2,3.2,3.2,16.925359,3.2,1149.151761,6.373706,3.2,3.2,3.2,3.2,26.603585,3.20000,60.512609,3.2,325.398968,15647.940247,1943.663852,25.105166,25.342591,24.074274,3.2,3.2,2.000000
2,K024,1902349,0,0,86,0,0,0,2,0,0,250.000000,269,6.50,1073,2,81,4.0,12,2.5,57.921345,6617.937597,17.636285,16.788346,9.765625,3.802534,NaN,5.767947,3.200000,32.800163,8.791588,3.200000,62.765297,3.867359,3.200000,433.780612,3.200000,4.018996,3.2,14.035815,3.2,3.2,3.2,23.676270,3.2,67.058080,3.200000,3.2,3.2,3.2,3.2,12.288175,3.20000,3.200000,3.2,35.723941,2807.826476,243.983235,5.912140,23.698467,8.748703,3.2,3.2,2.000000
3,K043,3165531,0,0,58,1,500,0,2,0,0,200.000000,538,11.26,2392,13,47,14.0,25,4.3,292.713992,8254.956691,12.391890,40.705676,9.765625,3.802534,NaN,9.398526,3.200000,20.027641,3.200000,3.200000,40.928415,4.625713,3.200000,416.843721,3.200000,21.447348,NaN,36.754655,3.2,3.2,3.2,4.925806,3.2,205.470611,3.200000,3.2,3.2,3.2,3.2,11.174294,3.20000,24.517336,3.2,87.506323,3908.900383,637.958551,3.518690,14.870714,NaN,3.2,3.2,2.000000
4,K045,3253683,0,0,76,1,800,0,3,0,0,380.952381,372,6.43,1555,9,37,36.0,7,2.3,645.580358,27110.719567,69.344563,92.977880,9.765625,NaN,NaN,36.132460,3.992006,19.948654,9.554105,3.200000,16.046589,3.200000,3.200000,252.724913,38.574521,40.631309,3.2,946.005892,3.2,3.2,3.2,6.572539,3.2,1089.321448,3.200000,3.2,3.2,3.2,3.2,7.745387,33.27508,240.737287,3.2,179.661551,2575.467315,3495.474916,31.821785,100.036875,21.233261,3.2,3.2,2.000000


In [5]:
# ヘルパーファンクションでテキスト内の"\n"をスペースに変換する
def clean_header(text):
  text = text.strip() # 最初と最後のスペースをテキストからとる
  text = text.replace(" ", "-")
  text_n_newline = text.replace("\n", " ")
  return text_n_newline

In [6]:
# ヘッダーコラムのテキストをリストとして変換する
AE_FILD_header_list = AE_FILD_df.columns.values.tolist()
AE_FILD_header_list

['No',
 'ID',
 '90day mortality\nsurvival:0\ndeath:1',
 'sex\nmale:0\nfemale:1',
 'age ',
 'smoking status\nNever:0\nEx(quit smoking over a year ago):1\nCurrent:2',
 'Brinkman Index',
 'emphysema\n(-):0\n(+):1',
 'Baseline ILD\nUIP:0\nprobable UIP:1\nIndeterminate for UIP:2\nAlternative diagnosis:3',
 'Collagen disease\n(-):0\n(+):1',
 'IPAF\n(-):0\n(+):1',
 'P/F ratio',
 'LDH',
 'CRP',
 'KL-6',
 '\nBAL(neutro %)',
 '\nBAL(Lymph %)',
 '\nBAL(Eos %)',
 '\nBAL(M %)',
 '\nBAL(CD4/8)',
 'MMP-1',
 'MMP-7',
 'TGFB1',
 'TGFB2',
 'TGFB3',
 'EGF',
 'FGF-2',
 'EOTAXIN',
 'TGF-a',
 'G-CSF',
 'Flt-3L',
 'GM-CSF',
 'FRACTALKINE',
 'IFN-a2',
 'IFN-g',
 'GRO',
 'IL-10',
 'MCP-3',
 'IL-12P40',
 'MDC',
 'IL-12P70',
 'IL-13',
 'IL-15',
 'sCD40L',
 'IL-17',
 'IL-1RA',
 'IL-1a',
 'IL-9',
 'IL-1b',
 'IL-2',
 'IL-3',
 'IL-4',
 'IL-5',
 'IL-6',
 'IL-7',
 'IL-8',
 'IP-10',
 'MCP-1',
 'MIP-1a',
 'MIP-1b',
 'TNFa',
 'TNFb',
 'VEGF',
 'IL-36β']

In [7]:
# 上記のヘッダーリストにclean_header()を応用する
AE_FILD_header_list_cleaned = [clean_header(x) for x in AE_FILD_header_list]
print(AE_FILD_header_list_cleaned)

['No', 'ID', '90day-mortality survival:0 death:1', 'sex male:0 female:1', 'age', 'smoking-status Never:0 Ex(quit-smoking-over-a-year-ago):1 Current:2', 'Brinkman-Index', 'emphysema (-):0 (+):1', 'Baseline-ILD UIP:0 probable-UIP:1 Indeterminate-for-UIP:2 Alternative-diagnosis:3', 'Collagen-disease (-):0 (+):1', 'IPAF (-):0 (+):1', 'P/F-ratio', 'LDH', 'CRP', 'KL-6', 'BAL(neutro-%)', 'BAL(Lymph-%)', 'BAL(Eos-%)', 'BAL(M-%)', 'BAL(CD4/8)', 'MMP-1', 'MMP-7', 'TGFB1', 'TGFB2', 'TGFB3', 'EGF', 'FGF-2', 'EOTAXIN', 'TGF-a', 'G-CSF', 'Flt-3L', 'GM-CSF', 'FRACTALKINE', 'IFN-a2', 'IFN-g', 'GRO', 'IL-10', 'MCP-3', 'IL-12P40', 'MDC', 'IL-12P70', 'IL-13', 'IL-15', 'sCD40L', 'IL-17', 'IL-1RA', 'IL-1a', 'IL-9', 'IL-1b', 'IL-2', 'IL-3', 'IL-4', 'IL-5', 'IL-6', 'IL-7', 'IL-8', 'IP-10', 'MCP-1', 'MIP-1a', 'MIP-1b', 'TNFa', 'TNFb', 'VEGF', 'IL-36β']


In [8]:
AE_FILD_df.columns = AE_FILD_header_list_cleaned
AE_FILD_df.head()

,No,ID,90day-mortality survival:0 death:1,sex male:0 female:1,age,smoking-status Never:0 Ex(quit-smoking-over-a-year-ago):1 Current:2,Brinkman-Index,emphysema (-):0 (+):1,Baseline-ILD UIP:0 probable-UIP:1 Indeterminate-for-UIP:2 Alternative-diagnosis:3,Collagen-disease (-):0 (+):1,IPAF (-):0 (+):1,P/F-ratio,LDH,CRP,KL-6,BAL(neutro-%),BAL(Lymph-%),BAL(Eos-%),BAL(M-%),BAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,K003,558277,0,0,78,1,1600,0,1,0,0,291.666667,345,2.35,1599,41,38,3.0,13,3.5,664.773204,13313.791597,62.176805,40.438809,9.765625,4.450383,NaN,10.963133,3.200000,10.836599,3.200000,3.200000,33.265989,3.200000,3.200000,954.746558,3.200000,34.734508,3.2,81.530691,3.2,3.2,3.2,12.667984,3.2,395.799401,3.379747,3.2,3.2,3.2,3.2,14.451335,3.20000,23.016166,3.2,313.439925,2160.940947,1009.878650,28.906316,30.940577,4.483265,3.2,3.2,4.221958
1,K022,1848827,0,1,72,0,0,0,2,0,0,264.571429,277,5.65,1620,18,21,0.0,61,3.9,561.092206,12107.253258,83.356309,64.460473,9.765625,6.932559,NaN,15.963448,3.200000,53.038298,3.200000,4.635806,75.733522,6.710735,10.849171,2849.703534,9.801179,46.135602,3.2,60.021260,3.2,3.2,3.2,16.925359,3.2,1149.151761,6.373706,3.2,3.2,3.2,3.2,26.603585,3.20000,60.512609,3.2,325.398968,15647.940247,1943.663852,25.105166,25.342591,24.074274,3.2,3.2,2.000000
2,K024,1902349,0,0,86,0,0,0,2,0,0,250.000000,269,6.50,1073,2,81,4.0,12,2.5,57.921345,6617.937597,17.636285,16.788346,9.765625,3.802534,NaN,5.767947,3.200000,32.800163,8.791588,3.200000,62.765297,3.867359,3.200000,433.780612,3.200000,4.018996,3.2,14.035815,3.2,3.2,3.2,23.676270,3.2,67.058080,3.200000,3.2,3.2,3.2,3.2,12.288175,3.20000,3.200000,3.2,35.723941,2807.826476,243.983235,5.912140,23.698467,8.748703,3.2,3.2,2.000000
3,K043,3165531,0,0,58,1,500,0,2,0,0,200.000000,538,11.26,2392,13,47,14.0,25,4.3,292.713992,8254.956691,12.391890,40.705676,9.765625,3.802534,NaN,9.398526,3.200000,20.027641,3.200000,3.200000,40.928415,4.625713,3.200000,416.843721,3.200000,21.447348,NaN,36.754655,3.2,3.2,3.2,4.925806,3.2,205.470611,3.200000,3.2,3.2,3.2,3.2,11.174294,3.20000,24.517336,3.2,87.506323,3908.900383,637.958551,3.518690,14.870714,NaN,3.2,3.2,2.000000
4,K045,3253683,0,0,76,1,800,0,3,0,0,380.952381,372,6.43,1555,9,37,36.0,7,2.3,645.580358,27110.719567,69.344563,92.977880,9.765625,NaN,NaN,36.132460,3.992006,19.948654,9.554105,3.200000,16.046589,3.200000,3.200000,252.724913,38.574521,40.631309,3.2,946.005892,3.2,3.2,3.2,6.572539,3.2,1089.321448,3.200000,3.2,3.2,3.2,3.2,7.745387,33.27508,240.737287,3.2,179.661551,2575.467315,3495.474916,31.821785,100.036875,21.233261,3.2,3.2,2.000000


In [9]:
# まず機械学習に不必要なコラムをdrop()する
AE_FILD_df = AE_FILD_df.drop(columns=["No"])
AE_FILD_df.head()

,ID,90day-mortality survival:0 death:1,sex male:0 female:1,age,smoking-status Never:0 Ex(quit-smoking-over-a-year-ago):1 Current:2,Brinkman-Index,emphysema (-):0 (+):1,Baseline-ILD UIP:0 probable-UIP:1 Indeterminate-for-UIP:2 Alternative-diagnosis:3,Collagen-disease (-):0 (+):1,IPAF (-):0 (+):1,P/F-ratio,LDH,CRP,KL-6,BAL(neutro-%),BAL(Lymph-%),BAL(Eos-%),BAL(M-%),BAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,558277,0,0,78,1,1600,0,1,0,0,291.666667,345,2.35,1599,41,38,3.0,13,3.5,664.773204,13313.791597,62.176805,40.438809,9.765625,4.450383,NaN,10.963133,3.200000,10.836599,3.200000,3.200000,33.265989,3.200000,3.200000,954.746558,3.200000,34.734508,3.2,81.530691,3.2,3.2,3.2,12.667984,3.2,395.799401,3.379747,3.2,3.2,3.2,3.2,14.451335,3.20000,23.016166,3.2,313.439925,2160.940947,1009.878650,28.906316,30.940577,4.483265,3.2,3.2,4.221958
1,1848827,0,1,72,0,0,0,2,0,0,264.571429,277,5.65,1620,18,21,0.0,61,3.9,561.092206,12107.253258,83.356309,64.460473,9.765625,6.932559,NaN,15.963448,3.200000,53.038298,3.200000,4.635806,75.733522,6.710735,10.849171,2849.703534,9.801179,46.135602,3.2,60.021260,3.2,3.2,3.2,16.925359,3.2,1149.151761,6.373706,3.2,3.2,3.2,3.2,26.603585,3.20000,60.512609,3.2,325.398968,15647.940247,1943.663852,25.105166,25.342591,24.074274,3.2,3.2,2.000000
2,1902349,0,0,86,0,0,0,2,0,0,250.000000,269,6.50,1073,2,81,4.0,12,2.5,57.921345,6617.937597,17.636285,16.788346,9.765625,3.802534,NaN,5.767947,3.200000,32.800163,8.791588,3.200000,62.765297,3.867359,3.200000,433.780612,3.200000,4.018996,3.2,14.035815,3.2,3.2,3.2,23.676270,3.2,67.058080,3.200000,3.2,3.2,3.2,3.2,12.288175,3.20000,3.200000,3.2,35.723941,2807.826476,243.983235,5.912140,23.698467,8.748703,3.2,3.2,2.000000
3,3165531,0,0,58,1,500,0,2,0,0,200.000000,538,11.26,2392,13,47,14.0,25,4.3,292.713992,8254.956691,12.391890,40.705676,9.765625,3.802534,NaN,9.398526,3.200000,20.027641,3.200000,3.200000,40.928415,4.625713,3.200000,416.843721,3.200000,21.447348,NaN,36.754655,3.2,3.2,3.2,4.925806,3.2,205.470611,3.200000,3.2,3.2,3.2,3.2,11.174294,3.20000,24.517336,3.2,87.506323,3908.900383,637.958551,3.518690,14.870714,NaN,3.2,3.2,2.000000
4,3253683,0,0,76,1,800,0,3,0,0,380.952381,372,6.43,1555,9,37,36.0,7,2.3,645.580358,27110.719567,69.344563,92.977880,9.765625,NaN,NaN,36.132460,3.992006,19.948654,9.554105,3.200000,16.046589,3.200000,3.200000,252.724913,38.574521,40.631309,3.2,946.005892,3.2,3.2,3.2,6.572539,3.2,1089.321448,3.200000,3.2,3.2,3.2,3.2,7.745387,33.27508,240.737287,3.2,179.661551,2575.467315,3495.474916,31.821785,100.036875,21.233261,3.2,3.2,2.000000


In [10]:
# データセット内のNaN値をどうするか (まずはinputer transformで試してみる)
from sklearn.impute import SimpleImputer
AE_FILD_df.replace('NaN', np.NaN, inplace=True) # データフレーム内の"NaN"をnumpyのNaNと置き換える
imp = SimpleImputer(missing_values=np.NaN, strategy="median") # sklearnのSimpleImputerを使い、np.NaNをnull値として認識させる。加、strategy="median"はnp.NaNをその行のNaNを中央値と置き換えるための指定
AE_FILD_df_idf = pd.DataFrame(imp.fit_transform(AE_FILD_df))
AE_FILD_df_idf.columns = AE_FILD_df.columns # SimpleImputerは数値しか作成できない為、元のデータセットの行名を新しいデータセットに加るコード
df_null_val = list(AE_FILD_df_idf.isna().sum()) # データセット内にNaN値が無いかの確認
print(f"AE_FILD_df_idfデータフレーム内のNaN数：{df_null_val}")

AE_FILD_df_idfデータフレーム内のNaN数：[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [11]:
from collections import OrderedDict

columnname_dict = OrderedDict()
columnname_dict['90day-mortality'] = {"survival":0.0, "death":1.0}
columnname_dict['sex'] = {"male":0.0, "female":1.0}
columnname_dict['smoking-status'] = {"Never":0.0, 'Ex(quit smoking over a year ago)':1.0, "Current":2.0}
columnname_dict['emphysema'] = {"-":0.0, "+":1.0}
columnname_dict['Baseline-ILD'] =  {"UIP":0.0, "probable UIP":1.0, "Indeterminate for UIP":2.0, "Alternative diagnosis":3.0}
columnname_dict['Collagen-disease'] = {"-":0.0, "+":1.0}
columnname_dict['IPAF'] = {"-":0.0, "+":1.0}

for key,values in columnname_dict.items():
  temp_dict = {}
  for k,v in values.items():
    temp_dict[v] = k
    columnname_dict[key] = temp_dict

columnname_dict

OrderedDict([('90day-mortality', {0.0: 'survival', 1.0: 'death'}),
             ('sex', {0.0: 'male', 1.0: 'female'}),
             ('smoking-status',
              {0.0: 'Never',
               1.0: 'Ex(quit smoking over a year ago)',
               2.0: 'Current'}),
             ('emphysema', {0.0: '-', 1.0: '+'}),
             ('Baseline-ILD',
              {0.0: 'UIP',
               1.0: 'probable UIP',
               2.0: 'Indeterminate for UIP',
               3.0: 'Alternative diagnosis'}),
             ('Collagen-disease', {0.0: '-', 1.0: '+'}),
             ('IPAF', {0.0: '-', 1.0: '+'})])

In [12]:
# ヘルパーファンクションでテキスト内の記号と置き換える
def remove_header_map(text):
  for key in columnname_dict.keys():
    if key in text:
      text = key
  return text

# ヘッダーコラムのテキストをリストとして変換する
AE_FILD_header_list = AE_FILD_df_idf.columns.values.tolist()
AE_FILD_header_list_cleaned = [remove_header_map(x) for x in AE_FILD_header_list]
AE_FILD_df_idf.columns = AE_FILD_header_list_cleaned
AE_FILD_df_idf.head()

,ID,90day-mortality,sex,age,smoking-status,Brinkman-Index,emphysema,Baseline-ILD,Collagen-disease,IPAF,P/F-ratio,LDH,CRP,KL-6,BAL(neutro-%),BAL(Lymph-%),BAL(Eos-%),BAL(M-%),BAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,558277.0,0.0,0.0,78.0,1.0,1600.0,0.0,1.0,0.0,0.0,291.666667,345.0,2.35,1599.0,41.0,38.0,3.0,13.0,3.5,664.773204,13313.791597,62.176805,40.438809,9.765625,4.450383,6.246947,10.963133,3.200000,10.836599,3.200000,3.200000,33.265989,3.200000,3.200000,954.746558,3.200000,34.734508,3.2,81.530691,3.2,3.2,3.2,12.667984,3.2,395.799401,3.379747,3.2,3.2,3.2,3.2,14.451335,3.20000,23.016166,3.2,313.439925,2160.940947,1009.878650,28.906316,30.940577,4.483265,3.2,3.2,4.221958
1,1848827.0,0.0,1.0,72.0,0.0,0.0,0.0,2.0,0.0,0.0,264.571429,277.0,5.65,1620.0,18.0,21.0,0.0,61.0,3.9,561.092206,12107.253258,83.356309,64.460473,9.765625,6.932559,6.246947,15.963448,3.200000,53.038298,3.200000,4.635806,75.733522,6.710735,10.849171,2849.703534,9.801179,46.135602,3.2,60.021260,3.2,3.2,3.2,16.925359,3.2,1149.151761,6.373706,3.2,3.2,3.2,3.2,26.603585,3.20000,60.512609,3.2,325.398968,15647.940247,1943.663852,25.105166,25.342591,24.074274,3.2,3.2,2.000000
2,1902349.0,0.0,0.0,86.0,0.0,0.0,0.0,2.0,0.0,0.0,250.000000,269.0,6.50,1073.0,2.0,81.0,4.0,12.0,2.5,57.921345,6617.937597,17.636285,16.788346,9.765625,3.802534,6.246947,5.767947,3.200000,32.800163,8.791588,3.200000,62.765297,3.867359,3.200000,433.780612,3.200000,4.018996,3.2,14.035815,3.2,3.2,3.2,23.676270,3.2,67.058080,3.200000,3.2,3.2,3.2,3.2,12.288175,3.20000,3.200000,3.2,35.723941,2807.826476,243.983235,5.912140,23.698467,8.748703,3.2,3.2,2.000000
3,3165531.0,0.0,0.0,58.0,1.0,500.0,0.0,2.0,0.0,0.0,200.000000,538.0,11.26,2392.0,13.0,47.0,14.0,25.0,4.3,292.713992,8254.956691,12.391890,40.705676,9.765625,3.802534,6.246947,9.398526,3.200000,20.027641,3.200000,3.200000,40.928415,4.625713,3.200000,416.843721,3.200000,21.447348,3.2,36.754655,3.2,3.2,3.2,4.925806,3.2,205.470611,3.200000,3.2,3.2,3.2,3.2,11.174294,3.20000,24.517336,3.2,87.506323,3908.900383,637.958551,3.518690,14.870714,3.792897,3.2,3.2,2.000000
4,3253683.0,0.0,0.0,76.0,1.0,800.0,0.0,3.0,0.0,0.0,380.952381,372.0,6.43,1555.0,9.0,37.0,36.0,7.0,2.3,645.580358,27110.719567,69.344563,92.977880,9.765625,4.350069,6.246947,36.132460,3.992006,19.948654,9.554105,3.200000,16.046589,3.200000,3.200000,252.724913,38.574521,40.631309,3.2,946.005892,3.2,3.2,3.2,6.572539,3.2,1089.321448,3.200000,3.2,3.2,3.2,3.2,7.745387,33.27508,240.737287,3.2,179.661551,2575.467315,3495.474916,31.821785,100.036875,21.233261,3.2,3.2,2.000000


In [13]:
# for columns in AE_FILD_df_idf.columns:
#     for key,value in columnname_dict.items():
#       if columns == key:
#         AE_FILD_df_idf[columns].replace(value,inplace=True)
# AE_FILD_df_idf.head()

In [14]:
# 模擬データを作るため、DataSynthesizer ライブラリーの導入
!pip install DataSynthesizer
from DataSynthesizer.DataDescriber import DataDescriber
from DataSynthesizer.DataGenerator import DataGenerator
from DataSynthesizer.ModelInspector import ModelInspector
from DataSynthesizer.lib.utils import read_json_file, display_bayesian_network

In [15]:
import os
# 上で作成したPandasのデータフレームをDataSynthesizerが応用できるようにcsvに変換する
AE_FILD_df_idf.to_csv("AE_FILD_df_idf.csv", index=False)

# 模擬データが元のデータの類似するように模擬データの作成設定をcorrelated_attribute_modeにする
mode = 'correlated_attribute_mode'

# 模擬データの構造を説明するための情報ファイル等の指定
os.makedirs("./out") # 出力ディレクトリーの作成
os.makedirs(f"./out/{mode}") # 模擬データのモード名を元にサブ出力ディレクトリーの作成
description_file = f'./out/{mode}/description.json'
synthetic_data = f'./out/{mode}/sythetic_data.csv'

In [16]:
categorical_attributes = {}

# データフレーム内の0−9行の内age以外の行はカテゴリカルなため、模擬データを作成出来るようにカテゴリかルアトリビュートとして認識する
for columns in list(AE_FILD_df_idf.columns[:9]):
  if columns != 'ID' and columns != 'age':
    categorical_attributes[columns] = True
# データの要素を識別できるように、IDをデータのキーとして認識する
candidate_keys = {'ID': True}

# OrderedDictがちゃんと作成されているか確認するコード
for key, val in categorical_attributes.items():
  print(f"{key}: {val}")

90day-mortality: True
sex: True
smoking-status: True
Brinkman-Index: True
emphysema: True
Baseline-ILD: True
Collagen-disease: True


In [17]:
# ドメインサイズがこのしきい値よりも小さい場合、属性はカテゴリーに分類されます。
# 今回のデータでは一番多いカテゴリー要素が4だったので、閾値を4と指定する
threshold_value = 4

# 差分プライバシーのパラメーター。 これは、入力データセットの行を削除しても、exp（epsilon）の乗法差以上に同じ出力を取得する確率が変わらないことを大まかに意味します。 
# イプシロン値を増やすことで学習モデルのノイズを減らせます。 差分プライバシーをオフにするにはepsilon = 0を設定します。
epsilon = 1

# ベイジアンネットワーク内の親ノードの最大数、つまり、有向グラフでの入力エッジの最大数。
degree_of_bayesian_network = 3

# DataDescriberをインスタンス化する
describer = DataDescriber(category_threshold=threshold_value)

# データセットの統計を計算する
describer.describe_dataset_in_correlated_attribute_mode(dataset_file='./AE_FILD_df_idf.csv', 
                                                        epsilon=epsilon, 
                                                        k=degree_of_bayesian_network,
                                                        attribute_to_is_categorical=categorical_attributes,
                                                        attribute_to_is_candidate_key=candidate_keys)

================ Constructing Bayesian Network (BN) ================
Adding ROOT MCP-1
Adding attribute GRO
Adding attribute IL-1a
Adding attribute IL-12P40
Adding attribute IFN-a2
Adding attribute FGF-2
Adding attribute IL-15
Adding attribute EOTAXIN
Adding attribute MIP-1a
Adding attribute VEGF
Adding attribute TGFB1
Adding attribute IL-3
Adding attribute GM-CSF
Adding attribute IL-10
Adding attribute MIP-1b
Adding attribute smoking-status
Adding attribute Brinkman-Index
Adding attribute 90day-mortality
Adding attribute IL-5
Adding attribute IL-1b
Adding attribute IL-7
Adding attribute IL-36β
Adding attribute IL-9
Adding attribute TGFB3
Adding attribute IL-17
Adding attribute Collagen-disease
Adding attribute MCP-3
Adding attribute IPAF
Adding attribute TNFa
Adding attribute TGF-a
Adding attribute MMP-1
Adding attribute BAL(neutro-%)
Adding attribute IL-1RA
Adding attribute TGFB2
Adding attribute FRACTALKINE
Adding attribute sex
Adding attribute MDC
Adding attribute IFN-g
Adding attr

In [18]:
# データセットの統計を上で定義した出力ファイルに保存する
# その後に作成されたベイズネットワークを出力する
describer.save_dataset_description_to_file(description_file)
display_bayesian_network(describer.bayesian_network)

Constructed Bayesian network:
    GRO              has parents ['MCP-1'].
    IL-1a            has parents ['GRO', 'MCP-1'].
    IL-12P40         has parents ['GRO', 'IL-1a', 'MCP-1'].
    IFN-a2           has parents ['IL-1a', 'IL-12P40', 'GRO'].
    FGF-2            has parents ['IL-1a', 'IFN-a2', 'MCP-1'].
    IL-15            has parents ['IL-12P40', 'IFN-a2', 'MCP-1'].
    EOTAXIN          has parents ['IL-1a', 'IL-12P40', 'GRO'].
    MIP-1a           has parents ['IL-1a', 'EOTAXIN', 'MCP-1'].
    VEGF             has parents ['GRO', 'MIP-1a', 'MCP-1'].
    TGFB1            has parents ['IFN-a2', 'FGF-2', 'GRO'].
    IL-3             has parents ['IFN-a2', 'EOTAXIN', 'IL-1a'].
    GM-CSF           has parents ['EOTAXIN', 'VEGF', 'IFN-a2'].
    IL-10            has parents ['FGF-2', 'TGFB1', 'MCP-1'].
    MIP-1b           has parents ['FGF-2', 'MIP-1a', 'IL-1a'].
    smoking-status   has parents ['IFN-a2', 'IL-10', 'GRO'].
    Brinkman-Index   has parents ['GM-CSF', 'MIP-1b', 'EOTA

In [19]:
# 合成データセットで生成される症例数。
num_tuples_to_generate = 5000

# 実際に模擬データを作成するコード
generator = DataGenerator()

# ベイズモデルの応用（模擬データの作成）
# description_fileとして定義してあるJSONファイルにベイズモデルが学習したカテゴリカルアトリビュートの特徴が保存される
# num_tuples_to_generateは生成される症例数
generator.generate_dataset_in_correlated_attribute_mode(num_tuples_to_generate, description_file)

# 作成した模擬データをsynthetic_dataとして定義したcsvファイルに保存する
generator.save_synthetic_data(synthetic_data)

# pandasのデータフレームとして模擬データを読み込む
synthetic_df = pd.read_csv(synthetic_data)
synthetic_df.head()

,ID,90day-mortality,sex,age,smoking-status,Brinkman-Index,emphysema,Baseline-ILD,Collagen-disease,IPAF,P/F-ratio,LDH,CRP,KL-6,BAL(neutro-%),BAL(Lymph-%),BAL(Eos-%),BAL(M-%),BAL(CD4/8),MMP-1,MMP-7,TGFB1,TGFB2,TGFB3,EGF,FGF-2,EOTAXIN,TGF-a,G-CSF,Flt-3L,GM-CSF,FRACTALKINE,IFN-a2,IFN-g,GRO,IL-10,MCP-3,IL-12P40,MDC,IL-12P70,IL-13,IL-15,sCD40L,IL-17,IL-1RA,IL-1a,IL-9,IL-1b,IL-2,IL-3,IL-4,IL-5,IL-6,IL-7,IL-8,IP-10,MCP-1,MIP-1a,MIP-1b,TNFa,TNFb,VEGF,IL-36β
0,0,1.0,1.0,80.0,2.0,1520.0,1.0,2.0,1.0,0.0,380.298569,1554.0,9.397128,0,64.0,26.0,24.0,59.0,7.010242,3890.208914,43720.669207,65.614933,119.001030,16.451159,4.756411,18.387240,54.236785,3.200000,91.745913,9.486645,4.719226,80.031205,5.149696,8.211610,93.208937,35.731843,16.096999,3.236687,372.435060,3.2,3.757003,4.027094,10.634768,3.2,1552.889112,19.607815,3.2,6.594884,3.2,3.2,32.452654,22.794253,291.546060,4.846057,1455.666827,8181.946889,3362.075894,29.503441,144.727834,48.756890,3.2,36.316894,4.221958
1,1,1.0,0.0,59.0,0.0,400.0,0.0,0.0,0.0,0.0,241.364543,1919.0,13.157690,1,52.0,64.0,36.0,54.0,1.739045,3374.194147,4549.001928,50.684577,11.319757,18.821699,6.786883,6.577175,18.247352,3.992006,34.304921,4.041290,6.493830,140.886603,24.877076,20.496724,1308.496601,25.569176,41.752065,4.312532,158.197757,3.2,5.033266,3.443958,6.672058,3.2,60.165432,15.200315,3.2,18.471427,3.2,3.2,21.391444,31.756955,150.001157,5.758452,813.638965,8065.935666,4177.092717,174.560551,416.605759,30.680322,3.2,12.715458,2.000000
2,2,1.0,0.0,78.0,0.0,700.0,0.0,0.0,0.0,1.0,272.978924,1923.0,25.960343,2,22.0,35.0,22.0,16.0,9.794742,1460.708444,72375.351371,112.665874,169.609057,17.335952,5.622011,18.099821,15.818029,3.992006,118.516236,9.015132,5.170758,68.050537,8.292625,9.950216,2844.555136,11.538208,130.948507,3.997336,600.335193,3.2,12.143407,3.962142,3.936790,3.2,1211.017590,20.940609,3.2,5.992057,3.2,3.2,15.516196,16.887486,223.737028,7.902392,461.763724,17201.044859,3648.341916,115.328829,470.435879,18.979407,3.2,50.088339,2.297510
3,3,1.0,1.0,81.0,2.0,700.0,1.0,1.0,1.0,1.0,369.185015,2413.0,2.492744,3,2.0,80.0,18.0,77.0,3.315201,144.321387,39684.924789,20.232103,127.144332,20.587796,9.621206,17.858985,38.247486,3.200000,48.355870,7.059914,6.393875,116.842590,13.401009,17.682459,1572.382091,20.910288,116.388584,3.396603,166.730595,3.2,5.933021,4.624772,9.583386,3.2,1403.753316,26.591532,3.2,20.220377,3.2,3.2,21.702797,27.623424,296.203269,5.891844,1416.472882,4710.903800,3119.651826,170.124154,73.638450,43.692546,3.2,37.787405,2.000000
4,4,0.0,1.0,80.0,2.0,400.0,0.0,2.0,0.0,0.0,337.733219,2509.0,1.073383,4,70.0,55.0,13.0,3.0,2.063978,1357.837842,45097.321421,102.512828,100.797933,22.337798,5.812509,3.403644,29.396973,3.992006,168.352324,9.262108,8.182242,96.972618,8.924200,23.155298,2743.718060,31.825527,113.648996,3.910172,175.299086,3.2,4.219082,3.703981,18.607388,3.2,1028.693086,51.366422,3.2,12.357943,3.2,3.2,12.894204,19.598541,22.598894,4.795117,723.642799,3418.467689,2586.772674,97.194692,334.335088,44.787819,3.2,35.808766,4.221958


In [20]:
# 新しく作成した模擬データは上記で作成したsynthetic_df内にあるため、AE_FILD_df_idfにデータを足す
AE_FILD_idf_syn = pd.concat([AE_FILD_df_idf, synthetic_df]) 

In [21]:
# # ベイズネットワークを作成するためにデータの要素をストリングに変換したため、上記で作成したディクショナリーを元に数字に変換する
# for columns in AE_FILD_idf_syn.columns:
#     for key,value in columnname_dict.items():
#       if columns == key:
#         for sub_k, sub_v in value.items():
#           AE_FILD_idf_syn[columns].replace({sub_k:value}, inplace=True)
# AE_FILD_idf_syn.head()

In [22]:
# ! pip install ydata_synthetic
# # 機械学習モデルがデータを学習する為に使うトランスフォメーション層の導入
# from ydata_synthetic.preprocessing import *

# train_data = AE_FILD_df.loc[AE_FILD_df['90day mortality survival:0 death:1']==1].copy()

# data = transformations(train_data)

# print("Dataset info: Number of records - {} Number of variables - {}".format(train_data.shape[0], train_data.shape[1]))

In [23]:
# from sklearn.neighbors import KernelDensity
# from sklearn.model_selection import GridSearchCV

# # ガウス法に基づきまずデータセットの密度をnumpyで表すためbandwithの設定
# # このbandwithの値に基づきデータ全体のパターンを機械学習モデルが予測しようとするため、機械学習モデルが勝手に最適bandwithを見つけてくれるものの、初期設定は重要
# bandwith_params = {'bandwidth': np.arange(0.01,1,0.05)} # [0.01 から１]の間でステップ0.05で値を作成
# # モデルをスキャンして最適パラメーターを見つける機械学習モデル（模擬データを作成するのに最適）
# grid_search = GridSearchCV(KernelDensity(), bandwith_params)
# # Aは従来の患者データ。fit()することで上記の機械学習モデルをトレーニングしてデータ内のパターンを学習してもらう
# grid_search.fit(A)
# kde = grid_search.best_estimator_

# # rand_stateの定義（rand_stateは新しいデータを作成する際のseedの定義）
# rand_state = 11
# # トレーニングしたモデルを使い新しい症例を20例作成してもらう
# new_data = kde.sample(20, random_state = rand_state)

# #　新しい症例を元のデータに加る
# A_updated = np.concatenate((A, new_data), axis=0)

In [24]:
# 今回の機械学習に使うライブラリーはsklearnのRF解析
# sklearnが機械学習モデルを適応できるようにデータをnumpyのarrayに変換する
A = AE_FILD_idf_syn.values #.valuesはnumpy arrayを出力する　
print(A)

[[5.58277000e+05 0.00000000e+00 0.00000000e+00 ... 3.20000000e+00
  3.20000000e+00 4.22195768e+00]
 [1.84882700e+06 0.00000000e+00 1.00000000e+00 ... 3.20000000e+00
  3.20000000e+00 2.00000000e+00]
 [1.90234900e+06 0.00000000e+00 0.00000000e+00 ... 3.20000000e+00
  3.20000000e+00 2.00000000e+00]
 ...
 [4.99700000e+03 1.00000000e+00 0.00000000e+00 ... 3.20000000e+00
  5.31855392e+01 2.00000000e+00]
 [4.99800000e+03 1.00000000e+00 0.00000000e+00 ... 3.20000000e+00
  1.83483758e+01 2.00000000e+00]
 [4.99900000e+03 1.00000000e+00 1.00000000e+00 ... 3.20000000e+00
  2.12554829e+01 2.00000000e+00]]


In [25]:
# 後で、乗算と除算ができるように、すべての値を浮動小数点に変換する
A = A.astype('float64')
print(A)

[[5.58277000e+05 0.00000000e+00 0.00000000e+00 ... 3.20000000e+00
  3.20000000e+00 4.22195768e+00]
 [1.84882700e+06 0.00000000e+00 1.00000000e+00 ... 3.20000000e+00
  3.20000000e+00 2.00000000e+00]
 [1.90234900e+06 0.00000000e+00 0.00000000e+00 ... 3.20000000e+00
  3.20000000e+00 2.00000000e+00]
 ...
 [4.99700000e+03 1.00000000e+00 0.00000000e+00 ... 3.20000000e+00
  5.31855392e+01 2.00000000e+00]
 [4.99800000e+03 1.00000000e+00 0.00000000e+00 ... 3.20000000e+00
  1.83483758e+01 2.00000000e+00]
 [4.99900000e+03 1.00000000e+00 1.00000000e+00 ... 3.20000000e+00
  2.12554829e+01 2.00000000e+00]]


In [26]:
# print(A_updated)
# print(A_updated.shape)

In [27]:
# 行数と列数を変数として保管する
NUM_ROWS, NUM_COLS = A.shape
print(f"\nこのデータセットの行数：{NUM_COLS}、列数：{NUM_ROWS}")


このデータセットの行数：63、列数：5060


In [28]:
# データセットのフィーチャー・ラベル定義

X_all = A[:,0:63].copy()  # X (フィーチャー) ... 全ての行
y_all = A[:,1].copy()    # y (ラベル) ... 全ての行, 2列目のみ

# Create a copy of X_all and y_all to apply rf analysis to later
X_all_rf = X_all.copy()
y_all_rf = y_all.copy()

print(f"X_all (フィーチャーのみ: 10行まで) \n {X_all[:10,:]}")
print(f"y_all (ラベルのみ) \n {y_all}") 

X_all (フィーチャーのみ: 10行まで) 
 [[5.58277000e+05 0.00000000e+00 0.00000000e+00 7.80000000e+01
  1.00000000e+00 1.60000000e+03 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00 2.91666667e+02 3.45000000e+02
  2.35000000e+00 1.59900000e+03 4.10000000e+01 3.80000000e+01
  3.00000000e+00 1.30000000e+01 3.50000000e+00 6.64773204e+02
  1.33137916e+04 6.21768045e+01 4.04388091e+01 9.76562500e+00
  4.45038295e+00 6.24694689e+00 1.09631327e+01 3.20000000e+00
  1.08365991e+01 3.20000000e+00 3.20000000e+00 3.32659886e+01
  3.20000000e+00 3.20000000e+00 9.54746558e+02 3.20000000e+00
  3.47345075e+01 3.20000000e+00 8.15306905e+01 3.20000000e+00
  3.20000000e+00 3.20000000e+00 1.26679837e+01 3.20000000e+00
  3.95799401e+02 3.37974706e+00 3.20000000e+00 3.20000000e+00
  3.20000000e+00 3.20000000e+00 1.44513353e+01 3.20000000e+00
  2.30161662e+01 3.20000000e+00 3.13439925e+02 2.16094095e+03
  1.00987865e+03 2.89063159e+01 3.09405765e+01 4.48326515e+00
  3.20000000e+00 3.20000000e+00 4.22195768e+

In [29]:
# フィーチャーの重要性を表すpythonのディクショナリー
# 最初の値は全ての行に1（データの内容を変えないように１で掛け算してもデータは変わらない）で指定しておけば、RFモデルがトレーニングする際に自動で更新してくれる
COL_WEIGHT_DICT = {}

for colname in AE_FILD_df_idf.columns[1:]: # 最初の行が出力データを表すため、フィーチャーの重みは最初の行以外を考慮する
  i = AE_FILD_df_idf.columns.get_loc(colname)
  weight = 1
  COL_WEIGHT_DICT[colname] = 1
  print(f"行名{colname}の重みを{weight}に指定しました")
  X_all[:,i] *= weight # そのコラム内のデータをweightで掛け算する

行名90day-mortalityの重みを1に指定しました
行名sexの重みを1に指定しました
行名ageの重みを1に指定しました
行名smoking-statusの重みを1に指定しました
行名Brinkman-Indexの重みを1に指定しました
行名emphysemaの重みを1に指定しました
行名Baseline-ILDの重みを1に指定しました
行名Collagen-diseaseの重みを1に指定しました
行名IPAFの重みを1に指定しました
行名P/F-ratioの重みを1に指定しました
行名LDHの重みを1に指定しました
行名CRPの重みを1に指定しました
行名KL-6の重みを1に指定しました
行名BAL(neutro-%)の重みを1に指定しました
行名BAL(Lymph-%)の重みを1に指定しました
行名BAL(Eos-%)の重みを1に指定しました
行名BAL(M-%)の重みを1に指定しました
行名BAL(CD4/8)の重みを1に指定しました
行名MMP-1の重みを1に指定しました
行名MMP-7の重みを1に指定しました
行名TGFB1の重みを1に指定しました
行名TGFB2の重みを1に指定しました
行名TGFB3の重みを1に指定しました
行名EGFの重みを1に指定しました
行名FGF-2の重みを1に指定しました
行名EOTAXINの重みを1に指定しました
行名TGF-aの重みを1に指定しました
行名G-CSFの重みを1に指定しました
行名Flt-3Lの重みを1に指定しました
行名GM-CSFの重みを1に指定しました
行名FRACTALKINEの重みを1に指定しました
行名IFN-a2の重みを1に指定しました
行名IFN-gの重みを1に指定しました
行名GROの重みを1に指定しました
行名IL-10の重みを1に指定しました
行名MCP-3の重みを1に指定しました
行名IL-12P40の重みを1に指定しました
行名MDCの重みを1に指定しました
行名IL-12P70の重みを1に指定しました
行名IL-13の重みを1に指定しました
行名IL-15の重みを1に指定しました
行名sCD40Lの重みを1に指定しました
行名IL-17の重みを1に指定しました
行名IL-1RAの重みを1に指定しました
行名IL-1aの重みを1に指定しました
行名IL-9の重みを1に指定し

In [30]:
# トレーニングデータとテストデータに入力データを分ける
# トレーニングデータ内のどのデータをどちらのデータセットに振り分けるかの指定をnpのrandom.permutationにより、毎回機械学習を行う際にランダムで決めてもらう
indices = np.random.permutation(len(y_all)) # ラベルの行のindexを記憶している変数
print(f"indices: {indices}")

# X_allとy_allに上記でnpが主力したランダムindexでデータセットの内容をシャッフルする （注：X_allとy_allは必ず同じindexでシャッフルしなくてはいけない）
X_all = X_all[indices]
y_all = y_all[indices]

print("置き換わったフィーチャーリスト：\n", X_all[:10,:])
print("置き換わったラベルリスト：\n", y_all)

indices: [4254 3114 3912 ... 2375 4455 2951]
置き換わったフィーチャーリスト：
 [[4.19400000e+03 0.00000000e+00 0.00000000e+00 8.40000000e+01
  2.00000000e+00 9.00000000e+02 1.00000000e+00 1.00000000e+00
  1.00000000e+00 0.00000000e+00 3.87527182e+02 3.80200000e+03
  2.54438684e+01 4.19400000e+03 7.20000000e+01 3.20000000e+01
  4.00000000e+00 7.50000000e+01 7.02506069e+00 4.69834036e+02
  4.28264607e+04 8.28264825e+01 2.95251496e+01 1.19444864e+01
  1.11662842e+01 5.80550897e+00 5.50304130e+01 3.20000000e+00
  2.28803529e+01 6.64031158e+00 7.80072834e+00 9.34425582e+01
  1.84379778e+01 2.42276698e+01 2.44060697e+03 1.13421331e+01
  1.12622484e+02 3.42687110e+00 4.90373208e+02 3.20000000e+00
  1.12612804e+01 3.50041271e+00 2.25295347e+01 3.20000000e+00
  1.22134764e+03 2.04905431e+01 3.20000000e+00 1.23180752e+01
  3.20000000e+00 3.20000000e+00 2.60874318e+01 2.46905390e+01
  9.58270984e+01 5.23618062e+00 1.32136244e+03 2.05369547e+03
  4.22991567e+03 6.91956841e+01 6.70273168e+01 3.96602277e+01
  3.200

In [31]:
# トレニーングデータとテストデータの振り分け
# 割合：トレーニング（8割）・テスト（2割）
NUM_ROWS = X_all.shape[0] # 列数
TEST_PERCENT = 0.2 # TEST ＋ TRAINING は必ず1にならなければいけない
TEST_SIZE = int(TEST_PERCENT * NUM_ROWS) 

X_test = X_all[:TEST_SIZE] # 0からTEST＿SIZEまでのデータ
y_test = y_all[:TEST_SIZE]

X_train = X_all[TEST_SIZE:] # TEST_SIZE以降のデータ
y_train = y_all[TEST_SIZE:]

print(f"トレーニング列数：{len(y_train)}、テスト列数：{len(y_test)}")

トレーニング列数：4048、テスト列数：1012


In [32]:
# RF解析に使うsklearnのライブラリーの導入
from sklearn import tree # 決定木モデル
from sklearn import ensemble # RF解析用のモデル
from sklearn.model_selection import cross_val_score # RF解析の最適解層をcross validationを使い見つけるためのモデル

In [33]:
# モデルの構築・モデルのトレーニング
# RF モデルの構築
# cross_valを使い、最適層を決めるんですが、事前に何階層を使えばいいかわからないため、適当に6階層決定木42と指定する
best_depth = 6
best_num_trees = 42
rforest_model = ensemble.RandomForestClassifier(max_depth=best_depth, n_estimators=best_num_trees) # ensemble.RandomForestClassifierからRF解析モデルを引っ張ってくる

# モデルの初期トレーニング
rforest_model.fit(X_train, y_train) # .fit()がトレーニングデータを使い、モデルをトレーニングしろというコマンド
print(f"RFモデルを{best_depth}層と{best_num_trees}決定木で作成しました")

RFモデルを6層と42決定木で作成しました


In [34]:
# 初期トレーニング後のモデルのパフォーマンス検証
# テストデータにrforest_modelに.predict()関数を応用し、RFモデルがラベルを何個正確に予測出来たかを検証する
predicted_labels = rforest_model.predict(X_test)
actual_labels = y_test

# ラベルをプリントして実際に予測vs.実際のラベルを見てみる
print(f"予測ラベル：{predicted_labels}")
print(f"実際ラベル: {actual_labels}")

# 正確性の出力
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"正確性は：{num_correct/total * 100}％でした")

予測ラベル：[0. 0. 0. ... 1. 1. 0.]
実際ラベル: [0. 0. 0. ... 1. 1. 0.]
正確性は：99.90118577075098％でした


In [35]:
# cross validationを使い、階層や決定木の数をチューニングしてみる（ハイパーパラメーター）
# 注：過適合/過剰適合のバランスを見極めながらハイパーパラメーターチューニングを行う
best_accuracy = 0
best_depth = 1
best_num_trees = 42

for ntrees in range(50,450,100):  # 決定木50から350の間（ステップ数：100)
    for d in range(1,20): # 階層1から20の間
        rforest_model = ensemble.RandomForestClassifier(max_depth=d, 
                                                        n_estimators=ntrees)
        cv_scores = cross_val_score( rforest_model, X_train, y_train, cv=5 ) # 5 はトレーニング・テストの割合が8割・2割ということを表す
        average_cv_accuracy = cv_scores.mean()  # cvスコアの平均値cv=5はスコアを五つ出すため、その平均値を見る
        if average_cv_accuracy >= best_accuracy: # cvスコアの平均値正確性が現在の最適正確性を上回った場合、現在のcvスコア平均値を出した決定木数・階層を最適とする
            best_accuracy = average_cv_accuracy
            best_depth = d
            best_num_trees = ntrees
        print(f"階層: {d:2d} 決定木: {ntrees:3d} cv スコア平均値: {average_cv_accuracy:7.4f}")

print()
print(f"最適層: {best_depth}, 最適決定木数： {best_num_trees}, 最高予測精度：{best_accuracy}")

階層:  1 決定木:  50 cv スコア平均値:  0.6932
階層:  2 決定木:  50 cv スコア平均値:  0.9988
階層:  3 決定木:  50 cv スコア平均値:  0.9840
階層:  4 決定木:  50 cv スコア平均値:  0.9995
階層:  5 決定木:  50 cv スコア平均値:  0.9990
階層:  6 決定木:  50 cv スコア平均値:  0.9993
階層:  7 決定木:  50 cv スコア平均値:  0.9990
階層:  8 決定木:  50 cv スコア平均値:  0.9993
階層:  9 決定木:  50 cv スコア平均値:  0.9990
階層: 10 決定木:  50 cv スコア平均値:  0.9993
階層: 11 決定木:  50 cv スコア平均値:  0.9993
階層: 12 決定木:  50 cv スコア平均値:  0.9993
階層: 13 決定木:  50 cv スコア平均値:  0.9990
階層: 14 決定木:  50 cv スコア平均値:  0.9990
階層: 15 決定木:  50 cv スコア平均値:  0.9990
階層: 16 決定木:  50 cv スコア平均値:  0.9995
階層: 17 決定木:  50 cv スコア平均値:  0.9990
階層: 18 決定木:  50 cv スコア平均値:  0.9990
階層: 19 決定木:  50 cv スコア平均値:  0.9993
階層:  1 決定木: 150 cv スコア平均値:  0.6107
階層:  2 決定木: 150 cv スコア平均値:  0.9815
階層:  3 決定木: 150 cv スコア平均値:  0.9993
階層:  4 決定木: 150 cv スコア平均値:  0.9993
階層:  5 決定木: 150 cv スコア平均値:  0.9990
階層:  6 決定木: 150 cv スコア平均値:  0.9993
階層:  7 決定木: 150 cv スコア平均値:  0.9990
階層:  8 決定木: 150 cv スコア平均値:  0.9995
階層:  9 決定木: 150 cv スコア平均値:  0.9990
階層: 10 決定木: 150 cv ス

In [36]:
# 上記で見つけた最適層・最適決定木数をもとにモデルの構築・モデルのトレーニング
rforest_model = ensemble.RandomForestClassifier(max_depth=best_depth, n_estimators=best_num_trees) # ensemble.RandomForestClassifierからRF解析モデルを引っ張ってくる

# モデルを再度トレーニングする
rforest_model.fit(X_train, y_train) # .fit()がトレーニングデータを使い、モデルをトレーニングしろというコマンド
print(f"RFモデルを{best_depth}層と{best_num_trees}決定木で作成しました")

RFモデルを17層と150決定木で作成しました


In [37]:
#  ラベル毎に予測ラベルが実際のラベルと一致してるか確認する関数
def compare_labels(predicted_labels, actual_labels):
    NUM_LABELS = len(predicted_labels)
    num_correct = 0
    
    for i in range(NUM_LABELS): 
        p = int(round(predicted_labels[i]))         # round protects from fp error 
        a = int(round(actual_labels[i]))
        result = "incorrect"
        if p == a:  # if they match,
            result = ""       # no longer incorrect
            num_correct += 1  # and we count a match!

        print(f"row {i:>3d} : {p} {a}   {result}") 

    print(f"\n正確性は：{num_correct/NUM_LABELS * 100}％でした")
    return num_correct

In [38]:
# 再度トレーニングした後のモデルのパフォーマンス検証
# テストデータにrforest_modelに.predict()関数を応用し、RFモデルがラベルを何個正確に予測出来たかを検証する
predicted_labels = rforest_model.predict(X_test)
actual_labels = y_test

# ラベルをプリントして実際に予測vs.実際のラベルを見てみる
print(f"予測ラベル：{predicted_labels}")
print(f"実際ラベル: {actual_labels}")

# 正確性の出力
compare_labels(predicted_labels, actual_labels)

予測ラベル：[0. 0. 0. ... 1. 1. 0.]
実際ラベル: [0. 0. 0. ... 1. 1. 0.]
row   0 : 0 0   
row   1 : 0 0   
row   2 : 0 0   
row   3 : 1 1   
row   4 : 1 1   
row   5 : 0 0   
row   6 : 1 1   
row   7 : 1 1   
row   8 : 1 1   
row   9 : 1 1   
row  10 : 0 0   
row  11 : 0 0   
row  12 : 1 1   
row  13 : 1 1   
row  14 : 1 1   
row  15 : 1 1   
row  16 : 0 0   
row  17 : 1 1   
row  18 : 1 1   
row  19 : 1 1   
row  20 : 0 0   
row  21 : 0 0   
row  22 : 0 0   
row  23 : 1 1   
row  24 : 1 1   
row  25 : 1 1   
row  26 : 0 0   
row  27 : 1 1   
row  28 : 0 0   
row  29 : 1 1   
row  30 : 1 1   
row  31 : 1 1   
row  32 : 1 1   
row  33 : 0 0   
row  34 : 1 1   
row  35 : 1 1   
row  36 : 1 1   
row  37 : 1 1   
row  38 : 0 0   
row  39 : 1 1   
row  40 : 1 1   
row  41 : 1 1   
row  42 : 1 1   
row  43 : 0 0   
row  44 : 0 0   
row  45 : 1 1   
row  46 : 1 1   
row  47 : 1 1   
row  48 : 0 0   
row  49 : 0 0   
row  50 : 1 1   
row  51 : 1 1   
row  52 : 0 0   
row  53 : 0 0   
row  54 : 1 1   
row 

1009

In [39]:
# フィーチャーの重要度を出力するコード
feature_importances = rforest_model.feature_importances_

feature_importances_dict = {}
for i, importance in enumerate(feature_importances):
  perc = importance * 100
  feature_importances_dict[AE_FILD_df_idf.columns[i]] = perc

sorted_dict = {}
sorted_keys = sorted(feature_importances_dict, key=feature_importances_dict.get)
for w in sorted_keys:
    sorted_dict[w] =  feature_importances_dict[w]
  
for keys in sorted_dict.keys():
  print(f"フィーチャー名：{keys:>12s}は出力データの結果に{sorted_dict[keys]:>7.2f}%関与しています")

フィーチャー名：    IL-12P70は出力データの結果に   0.00%関与しています
フィーチャー名：       IL-17は出力データの結果に   0.00%関与しています
フィーチャー名：        IL-9は出力データの結果に   0.00%関与しています
フィーチャー名：        IL-2は出力データの結果に   0.00%関与しています
フィーチャー名：        IL-3は出力データの結果に   0.00%関与しています
フィーチャー名：        TNFbは出力データの結果に   0.00%関与しています
フィーチャー名：Collagen-diseaseは出力データの結果に   0.04%関与しています
フィーチャー名：        IPAFは出力データの結果に   0.04%関与しています
フィーチャー名：         sexは出力データの結果に   0.05%関与しています
フィーチャー名：       TGF-aは出力データの結果に   0.05%関与しています
フィーチャー名：   emphysemaは出力データの結果に   0.06%関与しています
フィーチャー名：      IL-36βは出力データの結果に   0.09%関与しています
フィーチャー名：Baseline-ILDは出力データの結果に   0.11%関与しています
フィーチャー名：  BAL(Eos-%)は出力データの結果に   0.35%関与しています
フィーチャー名：BAL(Lymph-%)は出力データの結果に   0.37%関与しています
フィーチャー名：Brinkman-Indexは出力データの結果に   0.37%関与しています
フィーチャー名：        KL-6は出力データの結果に   0.39%関与しています
フィーチャー名：         ageは出力データの結果に   0.39%関与しています
フィーチャー名：        IL-8は出力データの結果に   0.40%関与しています
フィーチャー名：        IL-4は出力データの結果に   0.40%関与しています
フィーチャー名：       TGFB2は出力データの結果に   0.40%関与しています
フィーチャー名：  BAL(CD4/8)は出力データの結